* This notebook include several parts. Please __DO NOT__ run "__all__ the code". 
* Please __MODIFY File Dir__ before using.
* Author: Pengyu Xiao 2021.02
* Do not let out your API

In [1]:
import requests
import os
import json
import time
import pandas as pd

   ## Part 1  Authentication

In [2]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAABclLgEAAAAA%2Fka5GHCEomJcAuZigc8mgmQ5cTg%3DdQs7LinkRug5oIXVLel9YyG9kGi5lnr3NeWb0zWPgeF34zt3Pr"

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers ,params):
    response = requests.request("GET", url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

headers = create_headers(bearer_token)

## Part 2 User Profile Request
### 300 per15

In [9]:
# Simple Search Function
def create_url_pr_once(users, fields, rlimit=1000):
    usernames = "usernames="
    user_fields = "user.fields="
    if len(users) > rlimit:
        users=users[:rlimit]
    print("    ###    Search %d users' profiles    ###    " %len(users))

    for user in users:
        usernames = usernames + user + ","
    usernames = usernames[:-1]

    for field in fields:
        user_fields = user_fields + field +","
    user_fields = user_fields[:-1]

    url_pr_once = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url_pr_once

In [31]:
users = ["Tsinghua_Uni","sjokz","SpokespersonCHN","MUROO1_","sahin983540572","brahim36049212","rwsanimo","muhteremcavit","selmangl","senakursn","Edanurnazlix","Aysegul6347","ibrrahim_celik","AnnE34756701","Nergiz731","Calisanalisan","kinem_t","aysumavicicek","mertdmrc61","RonahiiiGencer","berkus","Aze2121","gulsumaltlr"]
fields = ["id","username","description","created_at","public_metrics","protected","location","profile_image_url"]


In [12]:
def create_url_pr_id(uid):
    url_pr_id =  "https://api.twitter.com/2/users/{}".format(uid)
    return url_pr_id

In [14]:
#url = create_url_pr_once(users, fields)
uid = 3381853252
url = create_url_pr_id(uid)
params = {}
json_response = connect_to_endpoint(url, headers, params)
print(json.dumps(json_response, indent=4, sort_keys=True))

200
{
    "data": {
        "id": "3381853252",
        "name": "\u9a6c\u8ba1",
        "username": "lxy1984000431"
    }
}


## Part 3  User Following and Followers Request 
### 15 per15

In [3]:
def create_url_fo_once(user_id, mode):
    # mode 0 -- followers, mode 1 -- followings
    url_fo_once =  "https://api.twitter.com/2/users/{}/follow".format(user_id)
    if mode:
        url_fo_once += "ing"
    else:
        url_fo_once += "ers"
    return url_fo_once

In [8]:
def req_url_fol(fol_user_id, mode, tlimit=15, nlimit=1000, startline=1, token=""):
    req_cnt = 0
    flag = 1
    params = {"max_results": 1000,
              "user.fields": "id,username,created_at,description,public_metrics,protected,location,profile_image_url,verified"
              }
    url = create_url_fo_once(fol_user_id, mode)
    
    while (flag):
        if token:
            params["pagination_token"] = token
        print(params)
        fol_response = connect_to_endpoint(url, headers, params)
        req_cnt += 1
    
        # Data Storage
        fdata = fol_response['data']
        fmeta = fol_response['meta']
        df = pd.DataFrame(columns=['id','username','created_at','description','public_metrics','protected','location','profile_image_url','verified'])
        for fjson in fdata:
            df.loc[startline] = fjson
            startline += 1
        print("Line: ", startline)
        print(df)
        df.to_csv(r"C:\Codes\AnonClassify\ScrapeData\id{}_followers.csv".format(fol_user_id), mode="a+", index=True, header=True)
        
        print(fmeta)
        if "next_token" in fmeta:
            token = fmeta["next_token"]
            time.sleep(60)
        else:
            flag = 0

In [ ]:
user_id = 279390084
req_url_fol(user_id, 0)

{'max_results': 1000, 'user.fields': 'id,username,created_at,description,public_metrics,protected,location,profile_image_url,verified'}
200
Line:  1000
                      id         username                created_at  \
0    1293516592383393792          jogiorn  2020-08-12T11:56:21.000Z   
1    1357161173037244418  William28236199  2021-02-04T02:57:12.000Z   
2              430138323         cenkozge  2011-12-06T20:19:30.000Z   
3    1074667614100733952      maelstrom_t  2018-12-17T14:08:20.000Z   
4    1365254191766241280   highschoolstu_  2021-02-26T10:55:48.000Z   
..                   ...              ...                       ...   
995  1359960669035061248    78__46n_e___7  2021-02-11T20:21:25.000Z   
996           2867179175  TheArifShahriar  2014-11-08T10:56:20.000Z   
997  1333798135215259649       ClaraDela5  2020-12-01T15:40:44.000Z   
998  1299040495398727682    Hidra22148054  2020-08-27T17:46:29.000Z   
999           2354906164  elneutrinoveloz  2014-02-20T18:04:50.000Z

---------

#### Function Codes Running On Server

In [ ]:
def req_url_fol(fol_user_id, mode, ofile, tlimit=15, nlimit=1000, startline=1, token=""):
    req_cnt = 0
    flag = 1
    params = {"max_results": 1000,
              "user.fields": "id,username,created_at,description,public_metrics,protected,location,profile_image_url,verified"
              }
    url = create_url_fo_once(fol_user_id, mode)
    if mode:
        fstr = "following.csv"
    else:
        fstr = "followers.csv"
    fstr = "/root/xpy/Data/id_{}_".format(fol_user_id) + fstr
    
    outfile = open(ofile, "a")
    while (flag):
        if token:
            params["pagination_token"] = token
        print(params, file= outfile)
        fol_response = connect_to_endpoint(url, headers, params)
        req_cnt += 1
    
        # Data Storage
        fdata = fol_response['data']
        fmeta = fol_response['meta']
        df = pd.DataFrame(columns=['id','username','created_at','description','public_metrics','protected','location','profile_image_url','verified'])
        for fjson in fdata:
            df.loc[startline] = fjson
            startline += 1
        print("Line: ", startline, file=outfile)
        print(df, file=outfile)
        
        df.to_csv(fstr, mode="a+", index=True, header=True)
        
        print(fmeta, file=outfile)
        if "next_token" in fmeta:
            token = fmeta["next_token"]
            time.sleep(60)
        else:
            flag = 0
            
    print("End at line %d" %startline, file=outfile)
    outfile.close()

--------

### Test Code 20210226

In [28]:
import pandas as pd
fdata = ff['data']
df = pd.DataFrame(columns=['id','username','created_at','description','public_metrics','protected','location','profile_image_url','verified'])
i = 0
for fjson in fdata:
    df.loc[i] = fjson
    i += 1
df


,id,username,created_at,description,public_metrics,protected,location,profile_image_url,verified
0,1219758271814426624,herseyhiranda,2020-01-21T23:06:58.000Z,,"{'followers_count': 0, 'following_count': 3, '...",True,NaN,https://abs.twimg.com/sticky/default_profile_i...,False
1,521725547,timboohig,2012-03-11T22:11:21.000Z,,"{'followers_count': 137, 'following_count': 59...",False,NaN,https://pbs.twimg.com/profile_images/132484507...,False
2,1357720190612500482,rev23471956,2021-02-05T15:58:33.000Z,,"{'followers_count': 3, 'following_count': 53, ...",False,NaN,https://pbs.twimg.com/profile_images/135772533...,False
3,1667226674,kayipxw,2013-08-13T08:32:18.000Z,Ne var,"{'followers_count': 235, 'following_count': 16...",True,Türkiye,https://pbs.twimg.com/profile_images/136415463...,False
4,1281359167865978881,reealperest,2020-07-09T22:46:57.000Z,Duck,"{'followers_count': 53, 'following_count': 78,...",False,NaN,https://pbs.twimg.com/profile_images/134798381...,False
...,...,...,...,...,...,...,...,...,...
995,273442896,svabbi98,2011-03-28T14:11:41.000Z,,"{'followers_count': 6, 'following_count': 72, ...",False,NaN,https://pbs.twimg.com/profile_images/337263608...,False
996,18190387,AlrightOk,2008-12-17T15:00:29.000Z,my avatar is oc.,"{'followers_count': 10, 'following_count': 589...",False,NYC,https://pbs.twimg.com/profile_images/854393729...,False
997,1077283764,ibhkarim,2013-01-10T18:13:17.000Z,@YaranNiger | @aidepartaction | Social Media M...,"{'followers_count': 8634, 'following_count': 1...",False,Niger 🇳🇪,https://pbs.twimg.com/profile_images/136473176...,False
998,1364646315804786690,OotsutsukiS,2021-02-24T18:40:23.000Z,"Sıkıntı, sıkıntı ve daha çok sıkıntı.","{'followers_count': 141, 'following_count': 20...",False,Her yer buradan iyidir,https://pbs.twimg.com/profile_images/136492075...,False


In [29]:
import sys
print("memory:{:.2f} KB".format(df.memory_usage().sum()/1024))

memory:78.12 KB


In [30]:
df.to_csv(r"C:\Codes\AnonClassify\ScrapeData\id279390084_followers.csv")

### Test End

----

----

### Divide Data Into 5K User Files

* This Part may be inconplete because of author's mistake
* However just try to run it

In [1]:
import pandas as pd
user_id = 279390084
file_dir = r"C:\Codes\AnonClassify\ScrapeData" + "\\id_{}_followers.csv".format(user_id)

In [10]:
file_dir
read_data = pd.read_csv(open(file_dir, 'r',encoding='UTF-8'))
# read_data

In [8]:
def cut_data(df, newfile_dir, thresh=5000):
    line  = df.shape[0]
    if line <= thresh:
        print("Small DataFrame")
    else:
        cnt = 1
        while cnt < line:
            try:
                d1 = df.iloc[cnt:cnt+thresh,:]
                fdir = newfile_dir + "_{}to{}.csv".format(cnt, cnt+thresh-1)
                d1.to_csv(fdir)
                print("Succeed to Store {} to {} data".format(cnt, cnt+thresh-1))
                cnt += thresh
            except:
                dl= df.iloc[cnt:line,:]
                fdir = newfile_dir + "_{}to{}.csv".format(cnt, line)
                d1.to_csv(fdir)
                print("Succeed to Store {} to {} data".format(cnt, line))
                cnt = line
            

In [11]:
newdir = r"C:\Codes\AnonClassify\ScrapeData\id_{}_followers".format(user_id) + "\\id_{}_followers".format(user_id)
if not os.path.exists(newdir)
    os.mkdir(newdir)
cut_data(read_data, newdir)

Succeed to Store 1 to 5000 data
Succeed to Store 5001 to 10000 data
Succeed to Store 10001 to 15000 data
Succeed to Store 15001 to 20000 data
Succeed to Store 20001 to 25000 data
Succeed to Store 25001 to 30000 data
Succeed to Store 30001 to 35000 data
Succeed to Store 35001 to 40000 data
Succeed to Store 40001 to 45000 data
Succeed to Store 45001 to 50000 data
Succeed to Store 50001 to 55000 data
Succeed to Store 55001 to 60000 data
Succeed to Store 60001 to 65000 data
Succeed to Store 65001 to 70000 data
Succeed to Store 70001 to 75000 data
Succeed to Store 75001 to 80000 data
Succeed to Store 80001 to 85000 data
Succeed to Store 85001 to 90000 data
Succeed to Store 90001 to 95000 data
Succeed to Store 95001 to 100000 data
Succeed to Store 100001 to 105000 data
Succeed to Store 105001 to 110000 data
Succeed to Store 110001 to 115000 data
Succeed to Store 115001 to 120000 data
Succeed to Store 120001 to 125000 data
Succeed to Store 125001 to 130000 data
Succeed to Store 130001 to 1350

---- 

## User Tweet Timeline Scraper(1500 per 15 for tweets/ 450 per 15 for mentions)

In [3]:
def create_url_tl_once(user_id):
    url_tl_once = "https://api.twitter.com/2/users/{}/tweets".format(user_id)
    return url_tl_once

In [6]:
def req_tl(user_id_tl, tlimit=1500, nlimit=100, start=1, token=""):
    tl_cnt=0
    flag=1
    params = {"max_results":nlimit,
             "tweet.fields": "context_annotations,conversation_id,created_at,entities,id,in_reply_to_user_id,referenced_tweets,lang,public_metrics,text"}
    url = create_url_tl_once(user_id_tl)
#     outfile = open(ofile, "a")
    
    sleeptime= 15*60/(tlimit / nlimit)
    
    #fstr = "/root/xpy/Data/id_{}_".format(user_id_tl) + "timeline.csv"
    while (flag):
        if token:
            params["pagination_token"] = token
        tl_response = connect_to_endpoint(url, headers, params)
        tl_cnt += 1
        print(tl_response)
        # Data Storage
        tdata = tl_response['data']
        tmeta = tl_response['meta']
        df = pd.DataFrame(columns=["context_annotations","conversation_id","created_at","entities","id","in_reply_to_user_id","referenced_tweets","lang","public_metrics","text"])
        for tjson in tdata:
            df.loc[start] = tjson
            start += 1
        print("Line: ", start)
        print(df)
#         print("Line: ", start, file=outfile)
#         print(df, file=outfile)
        
        #df.to_csv(fstr, mode="a+", index=False, header=True)
        
        #print(tmeta, file=outfile)
        print(tmeta)
        if "next_token" in tmeta:
            token = tmeta["next_token"]
            time.sleep(sleeptime)
        else:
            flag = 0
    print("End at line %d" %start)
            
#     print("End at line %d" %start, file=outfile)
#     outfile.close()
        
    # ofile,

In [8]:
user_id = 1233919379948965889
req_tl(user_id)

200
{'errors': [{'resource_id': '1233919379948965889', 'parameter': 'id', 'resource_type': 'user', 'section': 'data', 'title': 'Authorization Error', 'value': '1233919379948965889', 'detail': 'Sorry, you are not authorized to see the user with id: [1233919379948965889].', 'type': 'https://api.twitter.com/2/problems/not-authorized-for-resource'}]}


KeyError: 'data'

In [41]:
user_id = 279390084
url = create_url_tl_once(user_id)
params = {"tweet.fields": "context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,referenced_tweets,lang,public_metrics,text",
          "max_results": 5}
json_response = connect_to_endpoint(url, headers, params)
print(json_response)

200
{'data': [{'text': 'Brazil under Dictator Bolsonaro administration https://t.co/S4PS2FmdR7', 'lang': 'es', 'created_at': '2021-03-07T21:00:00.000Z', 'entities': {'annotations': [{'start': 0, 'end': 5, 'probability': 0.5952, 'type': 'Place', 'normalized_text': 'Brazil'}], 'urls': [{'start': 47, 'end': 70, 'url': 'https://t.co/S4PS2FmdR7', 'expanded_url': 'https://twitter.com/YourAnonNews/status/1368667774696951810/photo/1', 'display_url': 'pic.twitter.com/S4PS2FmdR7'}]}, 'conversation_id': '1368667774696951810', 'id': '1368667774696951810', 'public_metrics': {'retweet_count': 343, 'reply_count': 49, 'like_count': 2801, 'quote_count': 23}}, {'text': 'RT @ja1sharma: (1/n)Since, @Apple has fixed the bug overnight and declined to accept it!\nHere, I am sharing my not accepted Account Takeove…', 'lang': 'en', 'created_at': '2021-03-07T17:34:05.000Z', 'entities': {'mentions': [{'start': 3, 'end': 13, 'username': 'ja1sharma'}, {'start': 27, 'end': 33, 'username': 'Apple'}]}, 'conversation_

In [37]:
json_response

{'data': [{'public_metrics': {'retweet_count': 343,
    'reply_count': 49,
    'like_count': 2798,
    'quote_count': 23},
   'entities': {'annotations': [{'start': 0,
      'end': 5,
      'probability': 0.5952,
      'type': 'Place',
      'normalized_text': 'Brazil'}],
    'urls': [{'start': 47,
      'end': 70,
      'url': 'https://t.co/S4PS2FmdR7',
      'expanded_url': 'https://twitter.com/YourAnonNews/status/1368667774696951810/photo/1',
      'display_url': 'pic.twitter.com/S4PS2FmdR7'}]},
   'text': 'Brazil under Dictator Bolsonaro administration https://t.co/S4PS2FmdR7',
   'lang': 'es',
   'conversation_id': '1368667774696951810',
   'created_at': '2021-03-07T21:00:00.000Z',
   'id': '1368667774696951810'},
  {'public_metrics': {'retweet_count': 107,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'entities': {'mentions': [{'start': 3, 'end': 13, 'username': 'ja1sharma'},
     {'start': 27, 'end': 33, 'username': 'Apple'}]},
   'referenced_tweets': [{'t

----

## Get Full Text of Twitter (using referenced_tweet)

* Retweets will be truncated in response
* So use recent search

In [43]:
def create_url_tl_once(user_id):
    url_tl_once = "https://api.twitter.com/2/users/{}/tweets".format(user_id)
    return url_tl_once

In [46]:
user_id = 279390084
url = create_url_tl_once(user_id)
params = {"tweet.fields": "context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,referenced_tweets,lang,public_metrics,text",
          "max_results": 5,
          "expansions":"referenced_tweets.id"}
json_response = connect_to_endpoint(url, headers, params)
print(json_response)

200
{'data': [{'lang': 'en', 'created_at': '2021-03-09T03:10:28.000Z', 'id': '1369123392142651399', 'conversation_id': '1369123392142651399', 'entities': {'mentions': [{'start': 3, 'end': 13, 'username': 'donk_enby'}]}, 'referenced_tweets': [{'type': 'retweeted', 'id': '1369085054207541248'}], 'public_metrics': {'retweet_count': 30, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'text': "RT @donk_enby: I don't know and I don't care. I don't think there is one definition and I don't think there should be. \n\nAsk a 'hacker' wha…"}, {'lang': 'en', 'created_at': '2021-03-09T02:14:13.000Z', 'id': '1369109236907671552', 'conversation_id': '1369109236907671552', 'entities': {'mentions': [{'start': 3, 'end': 14, 'username': 'gemluvr257'}], 'urls': [{'start': 41, 'end': 64, 'url': 'https://t.co/PAgwmHlqXL', 'expanded_url': 'https://ddosecrets.substack.com/p/release-gableaks-70gb', 'display_url': 'ddosecrets.substack.com/p/release-gabl…', 'images': [{'url': 'https://pbs.twimg.com/news_im

In [47]:
json_response

{'data': [{'lang': 'en',
   'created_at': '2021-03-09T03:10:28.000Z',
   'id': '1369123392142651399',
   'conversation_id': '1369123392142651399',
   'entities': {'mentions': [{'start': 3,
      'end': 13,
      'username': 'donk_enby'}]},
   'referenced_tweets': [{'type': 'retweeted', 'id': '1369085054207541248'}],
   'public_metrics': {'retweet_count': 30,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'text': "RT @donk_enby: I don't know and I don't care. I don't think there is one definition and I don't think there should be. \n\nAsk a 'hacker' wha…"},
  {'lang': 'en',
   'created_at': '2021-03-09T02:14:13.000Z',
   'id': '1369109236907671552',
   'conversation_id': '1369109236907671552',
   'entities': {'mentions': [{'start': 3,
      'end': 14,
      'username': 'gemluvr257'}],
    'urls': [{'start': 41,
      'end': 64,
      'url': 'https://t.co/PAgwmHlqXL',
      'expanded_url': 'https://ddosecrets.substack.com/p/release-gableaks-70gb',
      'display_url

In [3]:
def create_url_tw_once(tw_id):
    url_tw_once = "https://api.twitter.com/2/tweets?ids={}".format(tw_id)
    return url_tw_once

In [4]:
tw_id = 1369123392142651399
url = create_url_tw_once(tw_id)
params = {"tweet.fields": "author_id,created_at,entities","expansions":"referenced_tweets.id"}
json_response = connect_to_endpoint(url, headers, params)
print(json_response)

200
{'data': [{'created_at': '2021-03-09T03:10:28.000Z', 'author_id': '279390084', 'referenced_tweets': [{'type': 'retweeted', 'id': '1369085054207541248'}], 'id': '1369123392142651399', 'text': "RT @donk_enby: I don't know and I don't care. I don't think there is one definition and I don't think there should be. \n\nAsk a 'hacker' wha…"}], 'includes': {'tweets': [{'created_at': '2021-03-09T00:38:07.000Z', 'author_id': '1213212165500325888', 'referenced_tweets': [{'type': 'quoted', 'id': '1369080087904202759'}], 'id': '1369085054207541248', 'text': 'I don\'t know and I don\'t care. I don\'t think there is one definition and I don\'t think there should be. \n\nAsk a \'hacker\' what the word means, you\'ll get a different answer each time. Here is mine:\n\n"someone with a creative, but skeptical attitude toward technology" https://t.co/eosefjErPi'}]}}


In [ ]:
def create_url_conv_once(conv_id):
    url_conv_once = "https://api.twitter.com/2/tweets/search/recent?query=conversation_id:{}".format(conv_id)
    return url_conv_once

conv_id =1369085054207541248
url = create_url_conv_once(conv_id)
params={}
json_response = connect_to_endpoint(url, headers, params)
print(json_response)

In [5]:
def create_url_tw_once(tw_id):
    url_tw_once = "https://api.twitter.com/2/tweets?ids={}".format(tw_id)
    return url_tw_once

In [7]:
import json
tw_id = 1369123392142651399
url = create_url_tw_once(tw_id)
params = {"tweet.fields": "author_id,created_at,entities,public_metrics,conversation_id","expansions":"referenced_tweets.id"}
json_response = connect_to_endpoint(url, headers, params)
print(json.dumps(json_response, sort_keys=True, indent=4, separators=(', ', ': ')))

200
{
    "data": [
        {
            "author_id": "279390084", 
            "conversation_id": "1369123392142651399", 
            "created_at": "2021-03-09T03:10:28.000Z", 
            "entities": {
                "mentions": [
                    {
                        "end": 13, 
                        "start": 3, 
                        "username": "donk_enby"
                    }
                ]
            }, 
            "id": "1369123392142651399", 
            "public_metrics": {
                "like_count": 0, 
                "quote_count": 0, 
                "reply_count": 0, 
                "retweet_count": 33
            }, 
            "referenced_tweets": [
                {
                    "id": "1369085054207541248", 
                    "type": "retweeted"
                }
            ], 
            "text": "RT @donk_enby: I don't know and I don't care. I don't think there is one definition and I don't think there should be. \n\nAsk a 'hacker' 

In [16]:
# Tweet Json Parser
json_response

if 'data' in json_response:
    jdata = json_response['data'][0]
    
#if 'includes' in json_response:

{'id': '1369123392142651399', 'entities': {'mentions': [{'start': 3, 'end': 13, 'username': 'donk_enby'}]}, 'created_at': '2021-03-09T03:10:28.000Z', 'referenced_tweets': [{'type': 'retweeted', 'id': '1369085054207541248'}], 'text': "RT @donk_enby: I don't know and I don't care. I don't think there is one definition and I don't think there should be. \n\nAsk a 'hacker' wha…", 'public_metrics': {'retweet_count': 35, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'author_id': '279390084'}


In [13]:
def create_url_conv_once(conv_id):
    url_conv_once = "https://api.twitter.com/2/tweets/search/recent?query=conversation_id:{}".format(conv_id)
    return url_conv_once

conv_id =1380062790736547840
url = create_url_conv_once(conv_id)
params={"tweet.fields": "author_id,created_at,entities,public_metrics,in_reply_to_user_id", "max_results": 100}
json_response = connect_to_endpoint(url, headers, params)
print(json_response)

200
{'data': [{'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'id': '1380359676102012932', 'author_id': '1162139270393417728', 'created_at': '2021-04-09T03:19:27.000Z', 'text': '@Michael90656953 台灣加油！', 'entities': {'mentions': [{'start': 0, 'end': 16, 'username': 'Michael90656953'}]}, 'in_reply_to_user_id': '1215965963092234240'}, {'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'id': '1380359380676272129', 'author_id': '1233117409336119296', 'created_at': '2021-04-09T03:18:16.000Z', 'text': '@Michael90656953 肌肉巨婴', 'entities': {'mentions': [{'start': 0, 'end': 16, 'username': 'Michael90656953'}]}, 'in_reply_to_user_id': '1215965963092234240'}, {'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'id': '1380359013313912832', 'author_id': '1267287668372541440', 'created_at': '2021-04-09T03:16:49.000Z', 'text': '@Michael90656953 🤣🤣🤣 是台湾吗？美国才对吧', 'entities':

In [14]:
json_response

{'data': [{'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'id': '1380359676102012932',
   'author_id': '1162139270393417728',
   'created_at': '2021-04-09T03:19:27.000Z',
   'text': '@Michael90656953 台灣加油！',
   'entities': {'mentions': [{'start': 0,
      'end': 16,
      'username': 'Michael90656953'}]},
   'in_reply_to_user_id': '1215965963092234240'},
  {'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'id': '1380359380676272129',
   'author_id': '1233117409336119296',
   'created_at': '2021-04-09T03:18:16.000Z',
   'text': '@Michael90656953 肌肉巨婴',
   'entities': {'mentions': [{'start': 0,
      'end': 16,
      'username': 'Michael90656953'}]},
   'in_reply_to_user_id': '1215965963092234240'},
  {'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'id': '1380359013313912832',
   'author_id': '1267287668372541

In [19]:
for fjson in json_response["data"]:
    print(fjson["public_metrics"]["retweet_count"])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
